## 사람인 API

In [9]:
import requests
import time

API_KEY = 'lMFsrk4jIe1FuCrTc5oeMuoQIydVwjjD2ys5MJ8989sGxYF3lS'

URL = "https://oapi.saramin.co.kr/job-search"

# 시니어 관련 키워드 리스트
keywords = ["노인", "시니어", "고령자", "실버", "60세"]

# 중복 제거를 위한 job id 기준 딕셔너리
job_results = {}

for keyword in keywords:
    params = {
        "access-key": API_KEY,
        "keywords": keyword,
        "ind_cd": "704,111,1001",       # 사회복지, 시설관리, 공공기관
        "job_mid_code": "20,10,7",      # 공공·복지, 서비스, 운전·운송
        "job_type": "2,5,14",           # 계약직, 알바, 파트타임
        "edu_lv": "0",                  # 학력무관
        "count": 100,
        "start": 0,
        "sort": "pd"
    }

    print(f"\n🔍 [{keyword}] 키워드로 검색 중...")

    try:
        response = requests.get(URL, headers={"Accept": "application/json"}, params=params)

        print("응답 Content-Type:", response.headers.get('Content-Type'))
        print("응답 상태 코드:", response.status_code)

        if response.status_code != 200:
            print(f"❌ 요청 실패: {response.status_code}")
            continue

        data = response.json()
        jobs = data.get("jobs", {}).get("job", [])

        for job in jobs:
            job_id = job.get("id")
            if not job_id or job_id in job_results:
                continue

            title = job.get("position", {}).get("title", "제목없음")
            company = job.get("company", {}).get("name", "회사없음")
            url = job.get("url", "링크없음")

            job_results[job_id] = {
                "title": title,
                "company": company,
                "url": url,
                "keyword": keyword
            }

        print(f"✅ {len(jobs)}개 공고 중 {len(job_results)}개 누적 저장됨.")
        time.sleep(1)  # API 서버 부하 방지를 위한 대기

    except Exception as e:
        print(f"🚨 예외 발생: {e}")

# 결과 출력
print(f"\n📦 최종 수집된 고유 공고 수: {len(job_results)}개")
for job in job_results.values():
    print(f"- [{job['keyword']}] {job['title']} - {job['company']} ({job['url']})")


🔍 [노인] 키워드로 검색 중...
응답 Content-Type: application/json;charset=UTF-8
응답 상태 코드: 200
✅ 100개 공고 중 100개 누적 저장됨.

🔍 [시니어] 키워드로 검색 중...
응답 Content-Type: application/json;charset=UTF-8
응답 상태 코드: 200
✅ 40개 공고 중 112개 누적 저장됨.

🔍 [고령자] 키워드로 검색 중...
응답 Content-Type: application/json;charset=UTF-8
응답 상태 코드: 200
✅ 11개 공고 중 122개 누적 저장됨.

🔍 [실버] 키워드로 검색 중...
응답 Content-Type: application/json;charset=UTF-8
응답 상태 코드: 200
✅ 26개 공고 중 139개 누적 저장됨.

🔍 [60세] 키워드로 검색 중...
응답 Content-Type: application/json;charset=UTF-8
응답 상태 코드: 200
✅ 25개 공고 중 161개 누적 저장됨.

📦 최종 수집된 고유 공고 수: 161개
- [노인] 봄날강서데이케어센터에서 사회복지사 선생님을 모집합니다. - 회사없음 (http://www.saramin.co.kr/zf_user/jobs/relay/view?rec_idx=51489732&utm_source=job-search-api&utm_medium=api&utm_campaign=saramin-job-search-api)
- [노인] 서울시니어타운 (서울 양천구 소재 요양원) 조리사 시간제근무 - 회사없음 (http://www.saramin.co.kr/zf_user/jobs/relay/view?rec_idx=51488014&utm_source=job-search-api&utm_medium=api&utm_campaign=saramin-job-search-api)
- [노인] 수원시 장안구 방문요양 요양보호사 구인 - 회사없음 (http://www.sarami

In [8]:
import requests
import time
import csv
import json

API_KEY = 'lMFsrk4jIe1FuCrTc5oeMuoQIydVwjjD2ys5MJ8989sGxYF3lS'
URL = "https://oapi.saramin.co.kr/job-search"
keywords = ["노인", "시니어", "고령자", "실버", "60세"]

all_jobs = []

job_results = {}

for keyword in keywords:
    params = {
        "access-key": API_KEY,
        "keywords": keyword,
        "ind_cd": "704,111,1001",# 산업/업종 코드표 中 사회복지, 시설관리·경비·용역, 정부·공공기관·공기업
        "job_mid_code": "20,10", # 직무/직업 코드표 中 공공·복지, 서비스
        "job_type": "2,5,14,16", # 근무형태 (계약직, 아르바이트, 파트, 기간제)
        "edu_lv": "0", #학력무관
        "count": 110,
        "start": 0,
        "sort": "pd",
        "fields": "expiration-date,posting-date"  # 명시적으로 포함
    }

    print(f"\n🔍 [{keyword}] 키워드로 검색 중...")

    try:
        response = requests.get(URL, headers={"Accept": "application/json"}, params=params)

        if response.status_code != 200:
            print(f"❌ 요청 실패: {response.status_code}")
            continue

        data = response.json()
        jobs = data.get("jobs", {}).get("job", [])
        all_jobs.extend(jobs)

        for job in jobs:
            job_id = job.get("id")
            if not job_id or job_id in job_results:
                continue

            job_results[job_id] = {
                "keyword": keyword,
                "title": job.get("position", {}).get("title", "제목 비어있음"),
                "company": job.get("company", {}).get("detail", {}).get("name", "회사 비어있음"),
                "location": job.get("position", {}).get("location", {}).get("name", "지역 비어있음"),
                "employment_type": job.get("position", {}).get("job-type", {}).get("name", "형태 비어있음"),
                "industry": job.get("position", {}).get("industry", {}).get("name", "업종 비어있음"),
                "education": job.get("position", {}).get("required-education-level", {}).get("name", "학력 비어있음"),
                "experience": job.get("position", {}).get("experience-level", {}).get("name", "경력 비어있음"),
                "salary": job.get("salary", {}).get("name", "급여 비어있음"),
                "posting": job.get("posting-date", "게시일 비어있음"),
                "expiration": job.get("expiration-date", "마감일 비어있음"),
                "url": job.get("url", "링크 비어있음")
            }

        print(f"✅ {len(jobs)}개 공고 중 {len(job_results)}개 누적 저장됨.")
        time.sleep(1)

    except Exception as e:
        print(f"🚨 예외 발생: {e}")

# CSV 파일로 저장
## 전체 저장
filename = "raw_senior_jobs.json"
with open(filename, "w", encoding="utf-8") as f:
    json.dump(all_jobs, f, ensure_ascii=False, indent=2)

print(f"\n📂 전체 raw job 데이터 저장 완료: {filename}")

## 필터링 저장
filename = "senior_jobs_fixed.csv"
with open(filename, mode='w', newline='', encoding='utf-8-sig') as file:
    writer = csv.DictWriter(file, fieldnames=[
    "keyword", "title", "company", "location", "employment_type",
    "industry", "education", "experience", "salary", "posting", "expiration", "url"
    ])
    writer.writeheader()
    writer.writerows(job_results.values())

print(f"\n📂 CSV 저장 완료: {filename}")


🔍 [노인] 키워드로 검색 중...
✅ 110개 공고 중 110개 누적 저장됨.

🔍 [시니어] 키워드로 검색 중...
✅ 43개 공고 중 119개 누적 저장됨.

🔍 [고령자] 키워드로 검색 중...
✅ 12개 공고 중 130개 누적 저장됨.

🔍 [실버] 키워드로 검색 중...
✅ 25개 공고 중 145개 누적 저장됨.

🔍 [60세] 키워드로 검색 중...
✅ 21개 공고 중 163개 누적 저장됨.

📂 전체 raw job 데이터 저장 완료: raw_senior_jobs.json

📂 CSV 저장 완료: senior_jobs_fixed.csv


In [ ]:
from flask import Flask, request, jsonify
import requests

app = Flask(__name__)

region_to_loc_cd = {
    "서울특별시 종로구": "101000",
    "서울특별시 중구": "102000",
    "서울특별시 강남구": "116800",
    "서울특별시 송파구": "117100"
}

@app.route("/api/recommend")
def recommend():
    region = request.args.get("region")
    loc_cd = region_to_loc_cd.get(region)

    if not loc_cd:
        return jsonify({"error": "지원하지 않는 지역입니다"}), 400

    API_KEY = 'lMFsrk4jIe1FuCrTc5oeMuoQIydVwjjD2ys5MJ8989sGxYF3lS'
    URL = "https://oapi.saramin.co.kr/job-search"
    params = {
        "access-key": API_KEY,
        "keywords": "노인",
        "loc_cd": loc_cd,
        "ind_cd": "704,111,1001",
        "job_mid_code": "20,10,7",
        "job_type": "2,5,14",
        "edu_lv": "0",
        "count": 100,
        "start": 0,
        "sort": "pd"
    }

    res = requests.get(URL, params=params)
    data = res.json()
    return jsonify(data)


## 고용24 API

In [17]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

# 고용24 API 키 입력
API_KEY = "47c2718d-5e98-45c2-afd2-8e06c05cf0d7"

# 요청 URL 및 파라미터 설정
BASE_URL = "https://www.work24.go.kr/cm/openApi/call/wk/callOpenApiSvcInfo210L01.do"
params = {
    "authKey": API_KEY,
    "callTp": "L",
    "returnType": "XML",
    "pfPreferential": "B",  # 고령자 우대 조건
    "display": "100",
    "startPage": "1",
    "sortOrderBy": "DESC"
}

# 요청 보내기
response = requests.get(BASE_URL, params=params)
root = ET.fromstring(response.content)

print(response.text)

# 데이터 파싱
jobs = []
print(f"찾은 공고 수: {len(root.findall('.//wantedList'))}")
for item in root.findall(".//wantedList"):
    job = {
        "회사명": item.findtext("corpNm", default=""),
        "채용제목": item.findtext("recrCont", default=""),
        "지역": item.findtext("region", default=""),
        "직종": item.findtext("occupation", default=""),
        "마감일": item.findtext("closeDt", default=""),
        "링크": item.findtext("recrUrl", default=""),
    }
    jobs.append(job)

# CSV로 저장
df = pd.DataFrame(jobs)
df.to_csv("고용24_고령자_채용공고.csv", index=False, encoding='utf-8-sig')
print("CSV 저장 완료!")


<?xml version='1.0' encoding='UTF-8'?>
<GO24>
  <error>개인회원은 사용할 수 없는 OPEN-API입니다.</error>
</GO24>

찾은 공고 수: 0
CSV 저장 완료!
